# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import gkey


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cityWeatherDF = pd.read_csv("../WeatherPy/Data/CityWeatherData.csv")
cityWeatherDF.head()

,Unnamed: 0,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,uvira,"Congo, The Democratic Republic of the",100,1579983502,94,-3.41,29.14,52.81,1.05
1,1,east london,South Africa,0,1579983502,85,-33.02,27.91,77.31,0.51
2,2,upington,South Africa,0,1579983502,21,-28.45,21.26,84.20,4.70
3,3,sao joao da barra,Brazil,100,1579983502,85,-21.64,-41.05,74.79,8.05
4,4,victoria,Seychelles,100,1579983319,95,22.29,114.16,62.01,1.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure 
gmaps.configure(api_key=gkey)

# get parts for heatmap
humidity = [float(h) for h in cityWeatherDF['Humidity']]
locations = cityWeatherDF[['Lat','Lng']]
locations = locations.rename(columns={'Lat':'Latitude','Lng':'Longitude'})

# add heatmap
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                    dissipating=False, max_intensity=max(humidity),
                                    point_radius=2)
fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# subset to only good weather
goodWeatherDF = cityWeatherDF[(cityWeatherDF['Max Temp'] > 65) & (cityWeatherDF['Max Temp'] < 80) & (cityWeatherDF['Humidity'] < 60) & (cityWeatherDF['Wind Speed'] < 8)]

goodWeatherDF = goodWeatherDF.reset_index()
goodWeatherDF

,index,Unnamed: 0,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,20,20,kapaa,United States,89,1579983404,59,22.08,-159.32,72.00,1.99
1,21,21,lompoc,United States,1,1579983504,48,34.64,-120.46,72.00,3.36
2,28,28,opuwo,Namibia,61,1579983526,36,-18.06,13.84,72.57,6.80
3,61,61,castro,Chile,0,1579983525,53,-24.79,-50.01,77.29,0.98
4,109,109,guerrero negro,Mexico,0,1579983598,43,27.98,-114.06,71.26,4.36
5,115,115,cukai,Malaysia,6,1579983620,10,4.25,103.42,75.00,1.01
6,135,135,paraiso,Mexico,23,1579983622,56,9.84,-83.87,77.00,5.84
7,152,152,niquero,Cuba,10,1579983624,47,20.04,-77.58,77.99,5.30
8,218,218,sarh,Chad,98,1579983674,11,9.14,18.39,68.99,4.65
9,223,223,obo,Central African Republic,76,1579983675,25,5.40,26.49,71.56,6.20


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# make hotel dataframe
hotel_df = goodWeatherDF
# make basic api url and params
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"location": "1,1",
          "rankby": "distance",
          "type": "hotel",
          "key": gkey}

In [6]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get location from df
    loc = f"{float(row['Lat'])},{float(row['Lng'])}"

    # add keyword to params dict
    params['location'] = loc

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'address'] = results[0]['vicinity']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: kapaa.
Closest hotel is Kapa’a Town Park.
------------
Retrieving Results for Index 1: lompoc.
Closest hotel is S B Sign Co.
------------
Retrieving Results for Index 2: opuwo.
Closest hotel is Bakery Restaurant.
------------
Retrieving Results for Index 3: castro.
Closest hotel is Eugênio Consertos e Instalações Eletricas.
------------
Retrieving Results for Index 4: guerrero negro.
Closest hotel is Generador Eolico.
------------
Retrieving Results for Index 5: cukai.
Closest hotel is Jangantengokje.
------------
Retrieving Results for Index 6: paraiso.
Closest hotel is New Times.
------------
Retrieving Results for Index 7: niquero.
Closest hotel is Vanier Liberaciones.
------------
Retrieving Results for Index 8: sarh.
Closest hotel is Hospital Central De Sarh.
------------
Retrieving Results for Index 9: obo.
Closest hotel is Poste Airport.
------------
Retrieving Results for Index 10: businga.
Missing field/result... skipping.
------------
Retrievin

In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in goodWeatherDF.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))